In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import string

In [0]:
pd.set_option('display.max_rows',15000)
pd.set_option('display.max_columns',500)

In [7]:
from google.colab import files
files.upload()

Saving p3.txt to p3.txt


{'p3.txt': b'ArticleTitle\tQuestion\tAnswer\tDifficultyFromQuestioner\tDifficultyFromAnswerer\tArticleFile\r\nAlessandro_Volta\tWas Alessandro Volta a professor of chemistry?\tAlessandro Volta was not a professor of chemistry.\teasy\teasy\tS10_set4_a10\r\nAlessandro_Volta\tWas Alessandro Volta a professor of chemistry?\tNo\teasy\thard\tS10_set4_a10\r\nAlessandro_Volta\tDid Alessandro Volta invent the remotely operated pistol?\tAlessandro Volta did invent the remotely operated pistol.\teasy\teasy\tS10_set4_a10\r\nAlessandro_Volta\tDid Alessandro Volta invent the remotely operated pistol?\tYes\teasy\teasy\tS10_set4_a10\r\nAlessandro_Volta\tWas Alessandro Volta taught in public schools?\tVolta was taught in public schools.\teasy\teasy\tS10_set4_a10\r\nAlessandro_Volta\tWas Alessandro Volta taught in public schools?\tYes\teasy\teasy\tS10_set4_a10\r\nAlessandro_Volta\tWho did Alessandro Volta marry?\tAlessandro Volta married Teresa Peregrini.\tmedium\tmedium\tS10_set4_a10\r\nAlessandro_Volt

In [0]:
df1=pd.read_table('P1.txt',sep='\t')
df2=pd.read_table('p2.txt',sep='\t')
df3=pd.read_table('p3.txt',sep='\t')

In [103]:
df1.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,S08_set3_a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,S08_set3_a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,S08_set3_a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,S08_set3_a4
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,S08_set3_a4


In [0]:
Dataset=df1.append([df2,df3])

In [105]:
Dataset.isna().sum()

ArticleTitle                   0
Question                      56
Answer                       716
DifficultyFromQuestioner    1178
DifficultyFromAnswerer       720
ArticleFile                    4
dtype: int64

In [125]:
Dataset

,QUEST,ANSW
0,was abraham lincoln the sixteenth president th...,yes
2,did lincoln sign the national banking act,yes
4,did his mother die pneumonia,
6,how many long was lincoln formal education,months
8,when did lincoln begin his political career,
10,what did the legal tender act establish,the united states note the first paper currenc...
12,who suggested lincoln grow beard,year old grace bedell
14,when did the gettysburg address argue that ame...,
16,did lincoln beat john breckinridge the election,yes
18,was abraham lincoln the first president the un...,


In [115]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1863 entries, 0 to 1456
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  1862 non-null   object
 1   Answer    1691 non-null   object
dtypes: object(2)
memory usage: 43.7+ KB


In [0]:
Dataset=Dataset.drop_duplicates(subset='Question')

In [0]:
Dataset=Dataset.dropna()

In [0]:
Dataset=Dataset.drop(labels=['ArticleTitle','DifficultyFromQuestioner','DifficultyFromAnswerer'],axis=1)
Dataset=Dataset.drop(labels=['ArticleFile'],axis=1)

In [0]:
def Question_stripping(Inpdata):
    cleanedArticle1=re.sub(r'[?|$|.@#=><|!]Â&*/',r' ',Inpdata)
    cleanedArticle2=re.sub(r'[^a-z A-Z]',r' ',cleanedArticle1)
    cleanedArticle3=cleanedArticle2.lower()
    cleanedArticle4=re.sub(r'\b\w{1,2}\b', ' ',cleanedArticle3)
    cleanedArticle5=re.sub(r'https?://\S+|www\.\S+',r' ',cleanedArticle4)
    cleanedArticle6=re.sub(r' +', ' ',cleanedArticle5)
    return(cleanedArticle6)

In [0]:
Dataset['QUEST']=Dataset['Question'].apply(Question_stripping)

In [0]:
def Answer_stripping(Inpdata):
    cleanedArticle1=re.sub(r'[?|$|.@#=><|!]Â&*/',r' ',Inpdata)
    cleanedArticle2=re.sub(r'[^a-z A-Z]',r' ',cleanedArticle1)
    cleanedArticle3=cleanedArticle2.lower()
    cleanedArticle4=re.sub(r'\b\w{1,2}\b', ' ',cleanedArticle3)
    cleanedArticle5=re.sub(r'https?://\S+|www\.\S+',r' ',cleanedArticle4)
    cleanedArticle6=re.sub(r' +', ' ',cleanedArticle5)
    return(cleanedArticle6)

In [0]:
Dataset['ANSW']=Dataset['Answer'].apply(Question_stripping)

In [0]:
Dataset=Dataset.drop(labels=['Question','Answer'],axis=1)

In [0]:
Dataset['ANSW']=Dataset['ANSW'].interpolate(method='bfill')

In [0]:
import nltk


In [44]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
def Tokenizer_question(Inpdata):
    stopwords_list = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(Inpdata)
    pos_tags = pos_tag(words)
    non_stopwords = [w for w in pos_tags if not w[0] in stopwords_list]
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    lemmas = []
    for w in non_punctuation:
      if w[1].startswith('J'):
            pos = wordnet.ADJ
      elif w[1].startswith('V'):
            pos = wordnet.VERB
      elif w[1].startswith('N'):
            pos = wordnet.NOUN
      elif w[1].startswith('R'):
            pos = wordnet.ADV
      else:
            pos = wordnet.NOUN
        
      lemmas.append(lemmatizer.lemmatize(w[0], pos))

      return lemmas


In [0]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=Tokenizer_question)
tfidf_matrix = tfidf_vectorizer.fit_transform(tuple(Dataset['QUEST']))

In [0]:
def ask_question(question):
  querry=tfidf_vectorizer.transform([question])
  similarity = cosine_similarity(querry, tfidf_matrix)
  max_similarity = np.argmax(similarity, axis=None)
  print('Your question:', question)
  print('Closest question found:', Dataset.iloc[max_similarity]['QUEST'])
  print('Similarity:',similarity[0, max_similarity])
  print('Answer:', Dataset.iloc[max_similarity]['ANSW'])

In [71]:
ask_question('When Abraham Lincoln started his political career')

Your question: When Abraham Lincoln started his political career
Closest question found: was abraham lincoln the sixteenth president the united states 
Similarity: 100.00%
Answer: yes


In [72]:
ask_question('have u done sex')

Your question: have u done sex
Closest question found: was abraham lincoln the sixteenth president the united states 
Similarity: 0.00%
Answer: yes


In [73]:
ask_question('How high are crime rates in Brazil')

Your question: How high are crime rates in Brazil
Closest question found: what the highest mountain romania 
Similarity: 100.00%
Answer: moldoveanu peak 


In [51]:
ask_question('Where was Nicola Tesla born')

Your question: Where was Nicola Tesla born
Closest question found: was abraham lincoln the sixteenth president the united states 
Similarity: 0.0
Answer: yes


In [59]:
ask_question('how many civilians died the embassy bombing	')

Your question: how many civilians died the embassy bombing	
Closest question found: how many long was lincoln formal education 
Similarity: 1.0
Answer:  months


In [57]:
ask_question('what the range lifespans the octopus')

Your question: what the range lifespans the octopus
Closest question found: what the range lifespans the octopus 
Similarity: 1.0
Answer: octopuses can live from six months five years 


In [52]:
ask_question('are pianos used western music')

Your question: are pianos used western music
Closest question found: are pianos used western music 
Similarity: 1.0
Answer: yes 


In [53]:

ask_question('what the middle pedal called grand pianos')

Your question: what the middle pedal called grand pianos
Closest question found: when was middle chinese used 
Similarity: 1.0
Answer: middle chinese was used during southern and northern dynasties and the sua tang and sang dynasties 


In [54]:
ask_question('how many arms does octopus have')

Your question: how many arms does octopus have
Closest question found: how many long was lincoln formal education 
Similarity: 1.0
Answer:  months


In [56]:

ask_question('who features violinist boyd tinsley')

Your question: who features violinist boyd tinsley
Closest question found: who features violinist boyd tinsley 
Similarity: 1.0
Answer: dave matthews band features violinist boyd tinsley 


In [60]:
ask_question('what violin called informally')

Your question: what violin called informally
Closest question found: does violin have four strings 
Similarity: 1.0
Answer: yes


In [61]:
ask_question('what does vitula mean')

Your question: what does vitula mean
Closest question found: what does vitula mean 
Similarity: 1.0
Answer: stringed instrument


In [62]:
ask_question('when was the reunification vietnam')

Your question: when was the reunification vietnam
Closest question found: when was the reunification vietnam 
Similarity: 1.0
Answer: the reunification vietnam occurred 


In [63]:
ask_question('does vietnamese have large number vowels')

Your question: does vietnamese have large number vowels
Closest question found: does vietnamese have comparatively large number vowels 
Similarity: 1.0
Answer: yes 


In [64]:

ask_question('what exception the rules vowel harmony compoun...')

Your question: what exception the rules vowel harmony compoun...
Closest question found: what exception the rules vowel harmony compound words have 
Similarity: 1.0
Answer:  compound words the vowels need not harmonize between the constituent words the compound 


In [66]:
ask_question('what are the patterns vowel harmony')

Your question: what are the patterns vowel harmony
Closest question found: what are the patterns vowel harmony 
Similarity: 1.0
Answer: vowels are characterised two features front back and rounded unrounded 


In [76]:

ask_question('what are the ways join groups nouns.')

Your question: what are the ways join groups nouns.
Closest question found:  what ways was adams opposed anderw hamilton 
Similarity: 1.0
Answer: hamilton wanted control the army differently than adams


In [78]:

ask_question('how many tones northern varieties have')

Your question: how many tones northern varieties have
Closest question found: how many long was lincoln formal education 
Similarity: 1.0
Answer:  months


In [80]:

ask_question('what age can zebra breed')

Your question: what age can zebra breed
Closest question found:  what age can zebra breed 
Similarity: 1.0
Answer: five six


In [82]:

ask_question('what zebras eat')

Your question: what zebras eat
Closest question found: what zebras eat 
Similarity: 1.0
Answer: mainly grass


In [83]:
ask_question('what areas the grevy zebras inhabit')

Your question: what areas the grevy zebras inhabit
Closest question found: what areas the grevy zebras inhabit 
Similarity: 1.0
Answer: semi arid grasslands ethiopia and northern kenya


In [84]:
ask_question('the xylophone instrument from what family')

Your question: the xylophone instrument from what family
Closest question found:  the xylophone musical instrument 
Similarity: 1.0
Answer: yes


In [85]:
ask_question('how the java and bali use xylophones')

Your question: how the java and bali use xylophones
Closest question found: how the java and bali use xylophones 
Similarity: 1.0
Answer:  gamelan ensembles


In [87]:

ask_question('which came first the xylophone the vibraphone')

Your question: which came first the xylophone the vibraphone
Closest question found: which came first the xylophone the vibraphone 
Similarity: 1.0
Answer: xylophone


In [91]:

ask_question('what are the ways join groups nouns')

Your question: what are the ways join groups nouns
Closest question found:  what ways was adams opposed anderw hamilton 
Similarity: 1.0
Answer: hamilton wanted control the army differently than adams


In [92]:
ask_question('what trumpet was the first allowed the christi..')

Your question: what trumpet was the first allowed the christi..
Closest question found:  the trumpet the musical instrument with the highest register the brass family 
Similarity: 1.0
Answer: yes


In [93]:
ask_question('what along the city western border.')

Your question: what along the city western border.
Closest question found: what along the city western border 
Similarity: 1.0
Answer: the danshui river borders taipei the west 


In [94]:
ask_question('what makes trumpet fully chromatic')

Your question: what makes trumpet fully chromatic
Closest question found: what makes more difficult for diving duck fly 
Similarity: 1.0
Answer: they are heavier


In [95]:
ask_question('what was built play both and keys')

Your question: what was built play both and keys
Closest question found: which building the site the german parliament 
Similarity: 1.0
Answer: the reichstag building the site the german parliament 


In [96]:
ask_question('what happened nineteen fifty seven')

Your question: what happened nineteen fifty seven
Closest question found: what happened 
Similarity: 1.0
Answer: avogadro had been recalled turin university


In [97]:

ask_question('what can augmented with fourth valve')

Your question: what can augmented with fourth valve
Closest question found: what can augmented with fourth valve 
Similarity: 1.0
Answer: the flugelhorn


In [98]:
ask_question('what are cymbals made	')

Your question: what are cymbals made	
Closest question found: what are cymbals made 
Similarity: 1.0
Answer: brass


In [99]:
ask_question('are cougars larger than jaguars	')

Your question: are cougars larger than jaguars	
Closest question found:  cougar also known mountain lion 
Similarity: 1.0
Answer:  cougar also known mountain lion 


In [126]:

ask_question('what shape trumpet bent into')

Your question: what shape trumpet bent into
Closest question found: what shape are leopard black ros settes east africa 
Similarity: 1.0
Answer: circular
